# Sistemas de recomendação

## O que são?

* sistemas que calculam e fornecem itens relevantes a um usuário baseado no conhecimento sobre o usuário, conteúdo e sobre as interações do usuário com os itens (tradução livre de uma definição encontrada em [[1]](https://www.manning.com/books/practical-recommender-systems))


## Importância hoje?

* anedotas
    1. youtube
    2. arxiv

## Exemplos:

* recomendações da Amazon após visualização do livro _Foundations of Natural Language Processing_:

![](resources/imgs/amazon_recommender.png)

* recomendações da Netflix para mim:

![](resources/imgs/netflix_recommender.png)

* **Mais??**

## Quando falamos de sistemas de recomendação...

... devemos definir:

* o quê?

* por quê?

* onde/como?

* quão personalizado?

## Efeitos indesejados

* invasão de privacidade

<img src="resources/imgs/target.png" width="500">

Fonte: [Business Insider](https://www.businessinsider.com/the-incredible-story-of-how-target-exposed-a-teen-girls-pregnancy-2012-2)

* espalhamento de desinformação

<img src="resources/imgs/youtube.png" width="300">

Fonte: [Twitter](https://twitter.com/PeterD_Adams/status/1089583479761788928). Leia mais no [NYT](https://www.nytimes.com/2018/03/10/opinion/sunday/youtube-politics-radical.html#click=https://t.co/cGE67FhlsI).

* quebra de confiança

<img src="resources/imgs/taboo.png" width="600">

Fonte: essas são as recomendações geradas pelo Taboola no final da página do [Business Insider](https://www.businessinsider.com/the-incredible-story-of-how-target-exposed-a-teen-girls-pregnancy-2012-2). Leia mais sobre esse tipo de recomendação em [Fortune](http://fortune.com/2016/11/01/outbrain-taboola/).

## Criando um sistema de recomendação

Nas atividades dessa aula, vamos usar uma modificação do dataset [goodbooks-10k](https://github.com/zygmuntz/goodbooks-10k), um dataset que contém as avaliações (_ratings_) para os 10 mil livros mais populares no site [goodreads](https://www.goodreads.com/).

Nosso objetivo será criar um sistema de recomendação assertivo; mais especificamente, queremos maximizar a leitura dos livros que recomendamos aos usuários.

### Carregando o dataset

Se você não fez o download do dataset ainda, rode a célula abaixo após descomentá-la (remova o `#`).

Para as próximas atividades, é importante carregar dois dos datasets disponíveis:

* `resources/data/reduced/ratings.csv`
* `resources/data/reduced/books.csv` (necessário caso queira ver que títulos estão sendo recomendados)

In [1]:
#!git clone -b only-reduced --single-branch https://github.com/cimarieta/goodbooks-10k.git resources/data/

In [2]:
import warnings

In [3]:
warnings.filterwarnings('ignore')

In [4]:
import pandas as pd

In [5]:
data_folder = 'resources/data/reduced'

In [6]:
user_ratings_df = pd.read_csv(f'{data_folder}/ratings.csv')

In [7]:
user_ratings_df.head(n=1)

,user_id,book_id,rating
0,75,3254,2


In [8]:
book_info_df = pd.read_csv(f'{data_folder}/books.csv')
book_info_df = book_info_df[['book_id', 'original_title', 'authors', 'original_publication_year']]

In [9]:
book_info_df.head(n=1)

,book_id,original_title,authors,original_publication_year
0,1,The Hunger Games,Suzanne Collins,2008.0


In [10]:
print('Informações sobre o dataset:\n')
print('* Quantidade de avaliações: {:,}'.format(len(user_ratings_df)))
print('* Quantidade de livros com avaliação: {:,}'.format(len(user_ratings_df['book_id'].unique())))
print('* Quantidade de usuários únicos: {: ,}'.format(len(user_ratings_df['user_id'].unique())))

Informações sobre o dataset:

* Quantidade de avaliações: 790,947
* Quantidade de livros com avaliação: 7,690
* Quantidade de usuários únicos:  5,000


### Filtro colaborativo

Há dois tipos de filtro colaborativo: o baseado em usuário (`user user`) e o baseado no conteúdo (`item item`).

<img src="resources/imgs/collaborative_filtering.png" width="400">

Fonte: [Practical Recommender Systems](https://www.manning.com/books/practical-recommender-systems)

#### **Atividade**: construção de um filtro colaborativo baseado em usuário

Etapas:

1. pré-processamento
    * normalizar dataset de ratings: para cada `rating`, será subtraída a média de `ratings` do usuário
    * construir uma matriz de usuários (`user_id`) por itens (`book_id`) - `ratings matrix`
    
2. cálculo das recomendações
    * encontrar os usuários similares para cada usuário (usando alguma métrica de similaridade, por ex., similaridade de cossenos)
    * construir uma função que, dado o id de um usuário, traz $n$ recomendações para ele

<img src="resources/imgs/user_item_collaborative_filtering_pipeline.png" width="600">

Fonte: [Practical Recommender Systems](https://www.manning.com/books/practical-recommender-systems)

Leia mais sobre filtros colaborativos em [um artigo de revisão do Grouplens](http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf).

** Tempo estimado para a atividade total: ** 45 minutos

#### Etapa 1: pré-processamento

**Etapa 1 - parte 1:** vamos normalizar o dataset `user_ratings_df`: para cada `rating`, será subtraída a média de `ratings` do usuário:

* construa um ` pandas.DataFrame` a partir do dataframe `user_ratings_df` com duas colunas: `user_id` e `mean_rating`, de forma que para cada `user_id`, tenhamos sua média de ratings na coluna `mean_rating`

In [11]:
mean_ratings_per_user_df = user_ratings_df \
    .groupby('user_id') \
    .agg({'rating': 'mean'}) \
    .reset_index() \
    .rename(columns={'rating': 'mean_rating'})

In [12]:
mean_ratings_per_user_df.head(n=2)

,user_id,mean_rating
0,7,3.835526
1,35,3.114943


In [13]:
user_ratings_df = user_ratings_df.merge(
    mean_ratings_per_user_df, on='user_id', how='inner')

In [14]:
user_ratings_df.head(n=2)

,user_id,book_id,rating,mean_rating
0,75,3254,2,3.496894
1,75,6777,5,3.496894


In [15]:
user_ratings_df['norm_rating'] = user_ratings_df['rating'] - user_ratings_df['mean_rating']

In [16]:
user_ratings_df.head(n=2)

,user_id,book_id,rating,mean_rating,norm_rating
0,75,3254,2,3.496894,-1.496894
1,75,6777,5,3.496894,1.503106


* vamos separar parte do nosso dataset como treino e parte dele como teste

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
train_indices, test_indices = train_test_split(range(len(user_ratings_df)))

* podemos verificar se a quantidade de usuários no dataset de treino é a mesma do dataset original

In [20]:
len(train_df['user_id'].unique()) == len(user_ratings_df['user_id'].unique())

True

**Etapa 1 - parte 2: ** vamos construir uma matriz de usuários por itens (`ratings_matrix`)

```python
# naive approach
user_items_matrix = pd.pivot_table(
    user_ratings_df,
    values='rating',
    index=['user_id'],
    columns=['book_id'],
    aggfunc=lambda x: x
).fillna(0).reset_index().values
``` 

<img src="resources/imgs/memory_error.png" width="800">

* para evitar erros de memória, vamos construir uma [matriz esparsa](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html) com os dados!

In [21]:
import numpy as np
from scipy.sparse import coo_matrix

In [22]:
unique_user_ids_df = user_ratings_df[['user_id']].drop_duplicates()
unique_user_ids_df['ref_user_id'] = np.arange(len(unique_user_ids_df))

In [23]:
unique_book_ids_df = user_ratings_df[['book_id']].drop_duplicates()
unique_book_ids_df['ref_book_id'] = np.arange(len(unique_book_ids_df))

* é importante conseguir identificar que linha da matriz (`id`) corresponde a que `user_id` e vice-versa, por isso vamos criar dois dicionários:

In [24]:
ref_id2user_id = unique_user_ids_df.set_index('ref_user_id')['user_id'].to_dict()
user_id2ref_id = {user_id: ref_id for ref_id, user_id in ref_id2user_id.items()}

* o mesmo vale para os `book_id`:

In [25]:
ref_id2book_id = unique_book_ids_df.set_index('ref_book_id')['book_id'].to_dict()
book_id2ref_id = {book_id: ref_id for ref_id, book_id in ref_id2book_id.items()}

* para facilitar os cálculos futuros, também vamos criar um dicionário com o rating médio de cada usuário:

In [26]:
def add_ref_ids_to_df(df):
    return df \
        .merge(unique_user_ids_df, on='user_id', how='inner') \
        .merge(unique_book_ids_df, on='book_id', how='inner')    

In [27]:
ref_user_id2mean_ratings = add_ref_ids_to_df(user_ratings_df)[['ref_user_id', 'mean_rating']] \
    .drop_duplicates() \
    .set_index('ref_user_id')['mean_rating'] \
    .to_dict()

In [101]:
user_ratings_df = add_ref_ids_to_df(user_ratings_df)

In [19]:
train_df, test_df = user_ratings_df.loc[train_indices], user_ratings_df.loc[test_indices]

In [29]:
unique_user_ids_len = len(unique_user_ids_df)
unique_book_ids_len = len(unique_book_ids_df)

In [30]:
def generate_user_interactions_matrix(df, data_col='rating'):
    data = df[data_col].values
    row = df['ref_user_id'].values
    col = df['ref_book_id'].values
    return coo_matrix(
        (data, (row, col)),
        shape=(unique_user_ids_len, unique_book_ids_len),
        dtype=np.float64
    ).tocsr()

In [31]:
unnormalized_user_items_matrix = generate_user_interactions_matrix(train_df)
unnormalized_test_user_items_matrix = generate_user_interactions_matrix(test_df)

In [32]:
user_items_matrix = generate_user_interactions_matrix(train_df, data_col='norm_rating')
test_user_items_matrix = generate_user_interactions_matrix(test_df, data_col='norm_rating')

* vamos salvar os dados para usar posteriormente

In [100]:
from sklearn.externals import joblib

In [103]:
# funciona no linux/mac
!mkdir -p resources/data/processed

In [106]:
joblib.dump(ref_id2user_id, 'resources/data/processed/ref_id2user_id.joblib')
joblib.dump(ref_id2book_id, 'resources/data/processed/ref_id2book_id.joblib')
joblib.dump(unnormalized_user_items_matrix, 'resources/data/processed/unnormalized_user_items_matrix.joblib')
joblib.dump(user_items_matrix, 'resources/data/processed/user_items_matrix.joblib')
joblib.dump(test_user_items_matrix, 'resources/data/processed/test_user_items_matrix.joblib')
joblib.dump((train_indices, test_indices), 'resources/data/processed/train_test_indices.joblib')
user_ratings_df.to_csv('resources/data/processed/user_ratings.csv', index=False)

In [33]:
user_items_matrix, test_user_items_matrix

(<5000x7690 sparse matrix of type '<class 'numpy.float64'>'
 	with 593210 stored elements in Compressed Sparse Row format>,
 <5000x7690 sparse matrix of type '<class 'numpy.float64'>'
 	with 197737 stored elements in Compressed Sparse Row format>)

#### Etapa 2: cálculo das recomendações

**Etapa 2 - parte 1:** vamos calcular a matriz de similaridades entre os usuários

Use a função [cosine_similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) para calcular a matriz que guardará os scores de similaridade para cada par de usuários. Aqui, a matriz de input para o cálculo das similaridades é a matriz não normalizada `unnormalized_user_items_matrix`.

Note que há liberdade para escolher a métrica de similaridade que será utilizada. Outra métrica bastante comum é a [correlação de Pearson](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.pearsonr.html). Veja outras métricas comuns [nesse post](http://dataaspirant.com/2015/04/11/five-most-popular-similarity-measures-implementation-in-python/).

** Tempo estimado para a atividade: ** 10 minutos

In [ ]:
similarities_matrix = ### 

* o tamanho da matriz de similaridades deve ser de (5000, 5000), pois 5000 é o número de usuários únicos em nosso dataset

In [ ]:
similarities_matrix.shape == (5000, 5000)

* crie uma função que dado um `ref_user_id` (ou seja, o índice do usuário na matriz) e um número $k$, retornemos uma tupla `(most_similar_users, similarities)` com os $k$ usuários (um vetor de `ref_user_id`) mais similares e com os scores de similaridade (um vetor com os scores). Note que queremos que essa tupla esteja ordenada de forma que tenhamos o usuário mais similar na primeira posição.

É importante que quando $k < 0$, retornemos **todos** os usuários.

In [37]:
def get_similar_users(ref_user_id, k=-1):
    ###

* teste de sanidade:
     o usuário mais próximo a um usuário $u_0$ sempre deve ser ele mesmo

In [38]:
u0 = 1
similar_users, similarities = get_similar_users(u0, k=5)
similar_users[0] == u0

True

* imprima os usuários e o score de similaridade para checar

In [39]:
for similar_user, similarity_score in zip(similar_users, similarities):
    print(f'ref_user_id: {similar_user:4d} \t - \t similarity_score: {similarity_score:5f}')

ref_user_id:    1 	 - 	 similarity_score: 1.000000
ref_user_id:  373 	 - 	 similarity_score: 0.236112
ref_user_id:   22 	 - 	 similarity_score: 0.225952
ref_user_id: 4628 	 - 	 similarity_score: 0.220277
ref_user_id:  432 	 - 	 similarity_score: 0.214147


**Etapa 2 - parte 2:** vamos construir uma função que recebe o id de um usuário e um número $n > 0$ e retorna $n$ recomendações para aquele usuário

Para um dado usuário `u`, devemos calcular as predições para todos itens que o usuário não avaliou (valor `0` na matriz `user_items_matrix`).

Ou seja, para cada item $i$ um dos itens não avaliados, vamos:

1. encontrar quais usuários avaliaram o item $i$
2. encontrar uma vizinhança de tamanho $k$ dentre os usuários que avaliaram aquele item (lembre-se de que se o usuário não avaliou o item, a célula correspondente será zero)
3. calcular a predição, que será a média dos ratings dessa vizinhança ponderada pelos valores de similaridade

Por exemplo:

Imagine que temos 4 usuários e 5 itens, de forma que tenhamos a seguinte matriz de ratings:

user_id | item_1 | item_2 | item_3 | item_4 | item_5
--------|--------|--------|--------|--------|--------|
user_1  |$r_{11}$|$r_{12}$|$r_{13}$|$r_{14}$|$r_{15}$|
user_2  |$r_{21}$|$r_{22}$|$r_{23}$|$r_{24}$|$r_{25}$|
user_3  |$r_{31}$|$r_{32}$|$r_{33}$|$r_{34}$|$r_{35}$|
user_4  |$r_{41}$|$r_{42}$|$r_{43}$|$r_{44}$|$r_{45}$|

Mas sabemos que alguns dos itens não foram avaliados:

user_id | item_1 | item_2 | item_3 | item_4 | item_5
--------|--------|--------|--------|--------|--------|
user_1  |        |$r_{12}$|$r_{13}$|$r_{14}$|        |
user_2  |$r_{21}$|$r_{22}$|        |$r_{24}$|$r_{25}$|
user_3  |$r_{31}$|$r_{32}$|        |$r_{34}$|$r_{35}$|
user_4  |$r_{41}$|        |$r_{43}$|$r_{44}$|$r_{45}$|

Então, para saber se devemos recomendar o `item_3` ao usuário `user_2`, vamos calcular:

$$r^{pred}_{23} = \mu_2 + \frac{S(u_1, u_2) \left(r_{23} - \mu_3 \right) +  S(u_4, u_2) \left(r_{43} - \mu_4 \right)}{|S(u_1, u_2)| + |S(u_4, u_2)|}$$

em que $\mu_i$ representa o rating médio do usuário `user_i` e $S(u_i, u_j)$ representa a similaridade entre os usuários `user_i` e `user_j`$.

Note que se $k=1$, o rating seria exatamente o rating dado pelo usuário mais próximo do `user_2`, entre os usuários `user_1` e `user_4`.

* encontrar quais usuários avaliaram cada um dos itens

Dica: use o método [nonzero](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.nonzero.html). Note que ele retornará uma tupla, com os índices das linhas e das colunas, respectivamente.

** Tempo estimado para a atividade: ** 5 minutos

In [40]:
rows, cols = ###

* crie uma função que recebe um `ref_book_id` e retorna os usuários que avaliaram aquele item

In [41]:
def get_users_who_rated_book(ref_book_id):
    ###

In [42]:
books_rated_by_users = {}
for ref_book_id in unique_book_ids_df.ref_book_id.values:
    books_rated_by_users[ref_book_id] = get_users_who_rated_book(ref_book_id)

* criar uma função que recebe o `user_id` ($\neq $ `ref_user_id`) e uma quantidade $n > 0$ e retorna os $n$ `book_id` ($\neq$ `ref_book_id`) recomendados

    Lembre-se de que podemos sempre utilizar os dicionários que criamos anteriormente:

    * `user_id2ref_id`: dado o `user_id`, retorna o `ref_user_id`
    * `ref_id2user_id`: dado o `ref_user_id`, retorna o `user_id`
    * `book_id2ref_id`: dado o `book_id`, retorna o `ref_book_id`
    * `ref_id2book_id`: dado o `ref_book_id`, retorna o `book_id`
    * `books_rated_by_users`: dado o `ref_book_id`, retorna um conjunto (`set`) de `ref_user_id` (pode ser um conjunto vazio)

* para auxiliar o cálculo, vamos criar a função `get_rating_predictions_from_CF`, que deverá receber um `ref_user_id`, uma lista de `ref_book_id` e um número $k > 0$, que é o tamanho da vizinhança a ser considerada. Essa função deve:
    1. descobrir quais são os usuários similares e quais os scores de similaridade (lembre-se de usar a função criada `get_similar_users` e de que aqui queremos retornar a lista com todos os usuários, sem especificar ainda a vizinhança)
    2. calcular o rating médio do usuário (pode-se usar o dicionário `ref_user_id2mean_ratings`)
    3. para cada livro `ref_book_id`: usar o rating médio, os scores de similaridade $S(i, \cdot)$ entre o usuário $i$ e os usuários similares, e os ratings **normalizados** ($r^{norm}_{ij}$ da matriz `user_items_matrix`) para calcular a predição para o user $i$ e livro $j$, segundo a fórmula:
        $$r^{pred}_{ij} = \mu_i + \frac{\sum\limits_{l=1}^{k} S(u_i, v_l) r_{lj}^{norm} }{\sum\limits_{l=1}^{k}|S(u_i, v_l)|}$$
        
        em que $\mu_i$ é o rating médio do usuário $i$, e $v_l$ representa um dos $k$ usuários mais próximos de $i$ que avaliaram o livro $j$. Caso o denominador seja zero, a fração será zero.
        
    4. retornar um dicionário no qual a chave é o `ref_book_id` e o valor é a predição
    
** Tempo estimado para a atividade: ** 15 minutos

In [ ]:
def get_rating_predictions_from_CF(ref_user_id, ref_book_ids_to_rate, k=5):
    similar_ref_user_ids, similarity_scores = get_similar_users(###)
    user_mean_rating = ###
    predictions = {}
    for ref_book_id in ref_book_ids_to_rate:
        predictions[ref_book_id] = ###
    return predictions

* com a função criada, podemos já fazer predições!

In [44]:
def make_appropriate_conversions_from_ref_id(func):
    def modified_func(user_id, *args, **kwargs):
        predictions = func(user_id2ref_id[user_id], *args, **kwargs)
        return [ref_id2book_id[ref_book_id] for ref_book_id in predictions]
    return modified_func

In [50]:
@make_appropriate_conversions_from_ref_id
def get_recommended_items_from_CF(ref_user_id, n=10, k=40):
    ref_book_ids_to_rate = get_unrated_items(ref_user_id)
    predictions = get_rating_predictions_from_CF(ref_user_id, ref_book_ids_to_rate, k=k)
    return sorted(predictions, key=lambda k: predictions[k], reverse=True)[:n]

* teste a recomendação para qualquer `user_id` e parâmetros $n$ e $k$. Você percebe alguma diferença?

In [51]:
predictions = get_recommended_items_from_CF(5447, n=10, k=10)

In [52]:
predictions

[70, 25, 27, 24, 23, 362, 133, 1616, 4230, 493]

## Fatorização de matrizes - construindo modelos com fatores latentes

Também podemos construir um sistema de recomendação usando técnicas de fatorização de matrizes. A ideia de fatorizar matrizes é reduzir a dimensionalidade e, consequentemente, a esparsidade das matrizes de interações, para gerar melhores recomendações para os usuários.

Veja mais sobre redução de dimensionalidade e modelos de fatores latentes nos seguintes vídeos do curso `Mining Massive Datasets`: [vídeo 1](https://www.youtube.com/watch?v=yLdOS6xyM_Q&list=PLLssT5z_DsK9JDLcT8T62VtzwyW9LNepV&index=46) e [vídeo 2](https://www.youtube.com/watch?v=E8aMcwmqsTg&list=PLLssT5z_DsK9JDLcT8T62VtzwyW9LNepV&index=55).

### SVD (Singular Value Decomposition)

A decomposição usando SVD de uma matriz $R$ é uma técnica que fatora a matriz de forma que:

$$R = U \cdot S \cdot V$$

sendo que $S$ é a matriz de valores singulares (autovalores) da matriz $R$.

O uso da técnica SVD em sistemas de recomendação parte da ideia de que com o SVD, podemos conseguir a melhor aproximação da matriz $R$ em uma dimensão menor $k$. Para saber mais, veja o [artigo do Grouplens](http://files.grouplens.org/papers/webKDD00.pdf).

Etapas:

1. imputação dos ratings faltando usando o rating médio (por coluna, ou seja, por livro) e normalização, subtraindo o rating médio **do usuário** (ou seja, por linha)
2. fatorização usando a função [svds do scipy.sparse.linalg](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.svds.html). Note que ela é **diferente** da função [svd do scipy.linalg](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.svd.html#scipy.linalg.svd);
3. cálculo das recomendações
    * para o usuário representado pela linha $i$ ($U_{i}$), os valores das predições de ratings para todos os itens serão calculados da seguinte forma:
        * primeiro, construímos a matriz usuário por tópicos: 
        $$M_{ut} = U_{i} \sqrt{S}$$

        * depois, construímos a matriz de tópicos por livros: 
        $$M_{tb} = \sqrt{S} V$$

        * e, finalmente, calculamos as predições (note que somamos o rating médio do usuário $\mu_i$): 
        $$preds = \mu_i +  M_{ut} M_{tb} $$

**Etapa 1:** imputação dos ratings faltando usando o rating médio (por coluna, ou seja, por livro) e normalização, subtraindo o rating médio do usuário (ou seja, por linha)

** Tempo estimado para a atividade: ** 10 minutos

* use a função [Imputer](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.Imputer.html#sklearn.preprocessing.Imputer) para substituir os zeros na matriz `unnormalized_user_items_matrix` pelo rating médio da respectiva coluna

* ponto de atenção: a biblioteca `scikit-learn` do env da Tera **não** é a versão estável: caso esteja usando outra versão mais nova, confira a [página do Imputer na versão estável do scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html)

In [ ]:
###

In [ ]:
user_items_matrix_for_SVD = ###

* com a matriz `user_items_matrix_for_SVD` pronta, subtraia dela a sua média (por usuário, ou seja, por linha)
    * para calcular a média, pode-se usar a função [numpy.mean](https://docs.scipy.org/doc/numpy/reference/generated/numpy.mean.html). Veja os exemplos e verifique se você deve usar `axis=0` ou `axis=1` como parâmetro
    * após calcular a média, a subtração precisa de um pequeno _truque_: devemos transformar em o vetor das médias em uma matriz; adicione `[:, np.newaxis]` ao final de seu vetor para fazer essa modificação

In [56]:
user_mean_rating_matrix = ###

In [57]:
user_items_matrix_for_SVD = ### 

**Etapa 2:** fatorização usando a função [svds do scipy.sparse.linalg](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.svds.html). Note que ela é **diferente** da função [svd do scipy.linalg](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.svd.html#scipy.linalg.svd)

Use a matriz gerada na etapa 1 como input do svd. Lembre-se que ele retornará três objetos: uma matriz $U$ (de usuários por $k$ fatores), um vetor com os $k$ maiores valores singulares e uma matriz $Vt$ (dos $k$ fatores pelos livros).

** Tempo estimado para a atividade: ** 5 minutos

In [ ]:
## Parâmetro para SVD
k = 20

In [ ]:
###

**Etapa 3:** cálculo das recomendações

* para auxiliar o cálculo, vamos criar a função `get_rating_predictions_from_SVD`, que deverá receber um `ref_user_id` e uma lista de `ref_book_id`. Essa função deve:
    1. calcular a aproximação dos ratings para a linha do usuário recebido $i$:
        * primeiro, construindo a matriz usuário por tópicos: 
        $$M_{ut} = U_{i} \sqrt{S}$$

        * depois, a matriz de tópicos por livros: 
        $$M_{tb} = \sqrt{S} V$$
        
        * e, finalmente, a matriz aproximada (1 linha e 7690 colunas):
        $$\tilde{R} = M_{ut} M_{tb} $$
        
    2. calcular o rating médio do usuário (pode-se usar o dicionário `ref_user_id2mean_ratings`)
    
    3. somar o rating médio a $\tilde{R}$
    
    4. retornar um dicionário no qual a chave é o `ref_book_id` e o valor é a predição, sendo que ele só deve conter os `ref_book_id` presentes na lista de `ref_book_id` recebida
    
** Tempo estimado para a atividade: ** 15 minutos

* essa matriz é a matriz $\sqrt{S}$ que  será usada nos cálculos

In [60]:
s_matrix_root_square = np.diag(np.sqrt(s))

* lembre-se de que para fazer as multiplicações de matrizes, você pode usar a função [numpy.dot](https://docs.scipy.org/doc/numpy-1.16.1/reference/generated/numpy.dot.html#numpy.dot)

In [61]:
def get_rating_predictions_from_SVD(ref_user_id, ref_book_ids_to_rate):
    M_ut = np.dot(###)
    M_tb = np.dot(###)
    approx_ratings_matrix = ###
    user_mean_rating = ###
    normalized_approx_ratings_matrix = ###
    predictions = {}
    for ref_book_id in ref_book_ids_to_rate:
        predictions[ref_book_id] = ###
    return predictions

* oba, agora já podemos fazer as predições!

In [61]:
@make_appropriate_conversions_from_ref_id
def get_recommended_items_from_SVD(ref_user_id, n=10):
    ref_book_ids_to_rate = get_unrated_items(ref_user_id)
    predictions = get_rating_predictions_from_SVD(ref_user_id, ref_book_ids_to_rate)
    return sorted(predictions, key=lambda k: predictions[k], reverse=True)[:n]

In [62]:
predictions = get_recommended_items_from_SVD(5447, n=10)

In [66]:
predictions

[9566, 1355, 1904, 2100, 3275, 9162, 8978, 6590, 3345, 8173]

### Bônus: comparação entre os dois modelos de recomendação

Que tal comparar a performance (em termos de [rmse](https://medium.com/human-in-a-machine-world/mae-and-rmse-which-metric-is-better-e60ac3bde13d)) dos dois modelos que criamos?

In [64]:
get_rating_predictions_from_CF(2801, [5810, 611, 50, 481, 1114, 30, 471], k=30)

{5810: 4.877098097106038,
 611: 4.945521922722298,
 50: 4.86506997527011,
 481: 5.028803979066535,
 1114: 4.9670518570273785,
 30: 4.875816993464053,
 471: 5.049323796307863}

In [65]:
get_rating_predictions_from_SVD(2801, [5810, 611, 50, 481, 1114, 30, 471])

{30: 4.408089210602784,
 50: 5.718953034913978,
 471: 5.625139588624251,
 481: 5.343427632937128,
 611: 5.113829598837039,
 1114: 5.195059237387463,
 5810: 4.96756347283006}

In [73]:
from sklearn.metrics import mean_squared_error

In [74]:
items_to_test_df = test_df[['ref_user_id', 'ref_book_id', 'rating']] \
    .groupby('ref_user_id') \
    .agg(lambda s: s.tolist()) \
    .reset_index()

In [75]:
from tqdm import tqdm

In [97]:
def get_rmse_from_test_df(get_rating_predictions_func, *args, **kwargs):
    preds = []
    true = []
    for row in tqdm(items_to_test_df.itertuples(), total=len(items_to_test_df)):
        rating_predictions_dict = get_rating_predictions_func(row.ref_user_id, row.ref_book_id, *args, **kwargs)
        preds += [rating_predictions_dict.get(ref_id, 0) for ref_id in row.ref_book_id]
        true += row.rating
    return np.sqrt(mean_squared_error(true, preds))

In [98]:
%%time
get_rmse_from_test_df(get_rating_predictions_from_CF, k=40)

100%|██████████| 5000/5000 [01:32<00:00, 53.90it/s]

CPU times: user 1min 32s, sys: 189 ms, total: 1min 33s
Wall time: 1min 32s


0.8717957658858932

In [99]:
%%time
get_rmse_from_test_df(get_rating_predictions_from_SVD)

100%|██████████| 5000/5000 [00:43<00:00, 114.02it/s]


CPU times: user 1min 16s, sys: 1min 24s, total: 2min 41s
Wall time: 43.9 s


0.8475938208369121